# Design Pattern 3: Feature Cross

> Cria um recurso sintético formado pela concatenação de dois ou mais recursos categóricos pata capturar a interação entre eles.

### Bibliotecas

In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, DenseFeatures
from tensorflow.feature_column import crossed_column, embedding_column

### Dados de teste

In [2]:
df = pd.read_csv('data/babyweight_train.csv')
df.dropna(inplace=True)
print(f'Linhas: {df.shape[0]} | Colunas: {df.shape[1]}')
df.head()

Linhas: 185665 | Colunas: 6


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,mother_race
0,7.749249,False,12,Single(1),40,1.0
1,7.561856,True,12,Single(1),40,2.0
2,7.187070,False,12,Single(1),34,3.0
3,6.375769,True,12,Single(1),36,2.0
5,6.000983,False,12,Single(1),39,2.0


### Design Pattern 3: Feature Cross

- Selecione as colunas
- Defina o tamanho dos buckets
- Crie a coluna cruzada combinando as características selecionadas
- Em seguida, podemos criar uma camada de incorporação e usar como entrada de uma rede neural

In [5]:
is_male = tf.feature_column.categorical_column_with_vocabulary_list('is_male', ['True', 'False', 'Unknown'])
plurality = tf.feature_column.categorical_column_with_vocabulary_list('plurality', ['Single', 'Twins', 'Triplets', 'Quadruplets', 'Quintuplets', 'Multiple'])

hash_bucket_size = 1000

gender_x_plurality = crossed_column(['is_male', 'plurality'], hash_bucket_size=hash_bucket_size)

crossed_feature = embedding_column(gender_x_plurality, dimension=2)

In [4]:
feature_columns = [crossed_feature]
X_train = df[:100].copy()


feature_layer_inputs = {}
for column in [is_male, plurality, crossed_feature]:
    feature_layer_inputs[column.name] = tf.keras.Input(shape=(1,), name=column.name, dtype=tf.string)

feature_layer = DenseFeatures([crossed_feature])(feature_layer_inputs)

x = Dense(128, activation='relu')(feature_layer)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=feature_layer_inputs, outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 is_male (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 is_male_X_plurality_embedd  [(None, 1)]                  0         []                            
 ing (InputLayer)                                                                                 
                                                                                                  
 plurality (InputLayer)      [(None, 1)]                  0         []                            
                                                                                                  
 dense_features (DenseFeatu  (None, 2)                    2000      ['is_male[0][0]',         